In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
print(tf.version.VERSION)

2.10.1


In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Initialize Model

In [100]:
from keras import layers
from keras import models

model = models.Sequential()
#feature learning portion
model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(50, 50, 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(256, (3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(512, (3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
#beginning of classification model
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.4))
#use sigmoid as binary classification
model.add(layers.Dense(1, activation='sigmoid'))

In [101]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_8 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 22, 22, 128)       73856     
                                                                 
 batch_normalization_9 (Batc  (None, 22, 22, 128)      512       
 hNormalization)                                                 
                                                      

In [102]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
tensorboard_dir = "logs/fit"

In [99]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
board = keras.callbacks.TensorBoard(log_dir=tensorboard_dir,
                                    histogram_freq=1,
                                    write_graph=True,
                                    write_images=True,
                                    update_freq='epoch',
                                    profile_batch=2,
                                    embeddings_freq=1)
lr_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=0.0001)

## Load images

In [11]:
train_dir = r"C:\Users\asaqu\Breast Cancer Detection Model\Model Data\Training"
val_dir = r"C:\Users\asaqu\Breast Cancer Detection Model\Model Data\Validation"
test_dir = r"C:\Users\asaqu\Breast Cancer Detection Model\Model Data\Testing"

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_data = train_datagen.flow_from_directory(train_dir,
                                         target_size = (50, 50),
                                         color_mode = 'grayscale',
                                         batch_size = 50,
                                         classes=['Negative','Positive'],
                                         class_mode = 'binary')

val_data = val_datagen.flow_from_directory(val_dir,
                                       target_size = (50, 50),
                                       color_mode = 'grayscale',
                                       batch_size = 50,
                                       classes=['Negative','Positive'],
                                       class_mode = 'binary')

test_data = test_datagen.flow_from_directory(test_dir,
                                       target_size = (50, 50),
                                       color_mode = 'grayscale',
                                       batch_size = 50,
                                       classes=['Negative','Positive'],
                                       class_mode = 'binary')
#flow from directory processes based on alphabetical order hence negative is 0 and positive is 1

Found 194266 images belonging to 2 classes.
Found 41629 images belonging to 2 classes.
Found 41629 images belonging to 2 classes.


## Set class weights

In [13]:
#calculate class weights
n_samples = 194266
n_classes = 2
n_sampleneg = 139329
n_samplepos = 54937

neg_weight = n_samples/(n_classes*n_sampleneg)

pos_weight = n_samples/(n_classes*n_samplepos)

print(neg_weight, pos_weight)

0.6971484759095379 1.768079800498753


## Train Model

In [103]:
#train model

with tf.device('/GPU:0'):
    result = model.fit(x=train_data,
                       batch_size = 50,
                       epochs=50,
                       validation_data=val_data,
                       class_weight ={0 : 0.697 ,
                                      1 : 1.768},
                       callbacks=[early_stop, lr_reduction]
                      )


Epoch 1/50
3886/3886 [==============================] - 88s 22ms/step - loss: 0.5219 - accuracy: 0.7669 - val_loss: 0.5130 - val_accuracy: 0.7570 - lr: 0.0010
Epoch 2/50
3886/3886 [==============================] - 87s 22ms/step - loss: 0.4801 - accuracy: 0.7900 - val_loss: 0.4476 - val_accuracy: 0.8015 - lr: 0.0010
Epoch 3/50
3886/3886 [==============================] - 86s 22ms/step - loss: 0.4594 - accuracy: 0.7992 - val_loss: 0.4449 - val_accuracy: 0.7855 - lr: 0.0010
Epoch 4/50
3886/3886 [==============================] - 85s 22ms/step - loss: 0.4440 - accuracy: 0.8061 - val_loss: 1.1601 - val_accuracy: 0.4416 - lr: 0.0010
Epoch 5/50
3886/3886 [==============================] - ETA: 0s - loss: 0.4279 - accuracy: 0.8117
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
3886/3886 [==============================] - 92s 24ms/step - loss: 0.4279 - accuracy: 0.8117 - val_loss: 0.5495 - val_accuracy: 0.7240 - lr: 0.0010
Epoch 6/50
3886/3886 [====================

In [104]:
model.save('breast_cancer_predictv4datawlabels.h5')

In [105]:
#save model results to evaluate
metrics = pd.DataFrame(model.history.history)
metrics.to_csv('model training results4.csv')

## Predict

In [118]:
predictions = model.predict(test_data, batch_size=50)
predictions

833/833 [==============================] - 19s 23ms/step


array([[1.7289824e-06],
       [2.9372478e-01],
       [7.2795373e-01],
       ...,
       [7.1698561e-02],
       [2.7570304e-01],
       [2.1577515e-03]], dtype=float32)

In [119]:
all_labels = []
for i in range(0,833):
    all_labels.append(test_data[i][1])
    

In [120]:
import numpy as np

all_labels = np.concatenate(all_labels, axis=0)
all_predictions = np.concatenate(predictions, axis=0)

In [121]:
all_labels.shape

(41629,)

In [122]:
results = pd.DataFrame({"Label" : all_labels,
                        "Prediction" : all_predictions})



In [123]:
results.shape

(41629, 2)

In [124]:
results.to_csv('predictions2.csv', index=False)

In [130]:
test_imgs, test_labels = next(test_data)

In [131]:
test_labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [132]:
first_batch_prediction = model.predict_generator(test_data, steps=1, verbose=0)

C:\Users\asaqu\AppData\Local\Temp\ipykernel_29272\1254772220.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  first_batch_prediction = model.predict_generator(test_data, steps=1, verbose=0)


In [133]:
first_batch_prediction

array([[8.1463680e-02],
       [2.4295032e-01],
       [2.6929730e-01],
       [4.3179470e-01],
       [2.6489824e-01],
       [1.3212118e-02],
       [1.7888506e-01],
       [7.1960196e-02],
       [7.3630572e-04],
       [8.4828045e-03],
       [2.4432458e-01],
       [3.1685910e-01],
       [2.7245793e-01],
       [4.3625609e-05],
       [4.4033553e-02],
       [1.6254438e-05],
       [5.4892892e-01],
       [4.1048539e-01],
       [2.2640441e-01],
       [4.5043617e-01],
       [8.7172937e-01],
       [1.0982441e-01],
       [8.6356038e-01],
       [5.1711470e-01],
       [2.8568524e-01],
       [2.6097611e-01],
       [6.9239557e-01],
       [5.2635748e-02],
       [5.7363551e-02],
       [2.8496835e-01],
       [4.8303057e-02],
       [2.9211659e-02],
       [4.8990082e-02],
       [7.4228980e-02],
       [9.7219265e-01],
       [5.9567499e-03],
       [9.8694277e-01],
       [2.2513363e-01],
       [3.9907983e-01],
       [3.4822318e-01],
       [9.6889549e-01],
       [8.612331

In [134]:
single_batch_labels = list(test_labels.flatten())
single_batch_predictions = np.concatenate(first_batch_prediction, axis=0)

In [135]:
single_batch_results = pd.DataFrame({"Label" : single_batch_labels,
                        "Prediction" : single_batch_predictions})

In [136]:
single_batch_results.to_csv('single_batch_result.csv')